In [8]:
'''
Load sentences from `kwdlc_knp` and names from `names`,
Generate sample and truth data for family_name/given_name/full_name and kannji/katakana/hiragana/roma respectively.
'''
from pathlib import Path
from typing import Dict, List
import pandas as pd
import datetime
import tqdm

def update_name(text: str, labels: List[List], name: str) -> List:
    '''
    labels are the locations of name in the text.
    update the names and labels with the input names and return the new text and labels.
    '''
    chars = list(text)
    labels = labels[:]
    new_labels = list()
    while labels:
        s, e = labels.pop(0)
        chars[s: e] = name
        diff = len(name) - (e - s)
        e += diff
        for l in labels:
            l[0] += diff
            l[1] += diff
        new_labels.append((s, e))
    return ''.join(chars), new_labels

def generate_data(sent_csv: Path, name_csv: Path, name_style: str) -> Dict[str, pd.DataFrame]:
    '''
    :param sent_csv: Path. Csv file with columns of id, text and labels
    :param name_csv: Path. Csv file with columns of name, rank, katakana, hiragana, roma
    :param name_style: str. {'kanji', 'katakana', 'hiragana', 'roma'}
    :return: pd.Dataframe.
        For sample data, the columns consist of
        - school_id (1)
        - created_at (YYYY-MM-DD HH:mm:SS)
        - updated_at (YYYY-MM-DD HH:mm:SS)
        - id
        - text
        For truth data (for evaluation), the columns consist of
        - id
        - text
        - labels
    '''
    # Load sent_csv
    sent_df = pd.read_csv(sent_csv, index_col="id")
    name_df = pd.read_csv(name_csv, keep_default_na=False)
    texts = list()
    labels = list()
    school_id = list()
    created_at, updated_at = list(), list()

    names = name_df[name_style]
    for _, row in tqdm.tqdm(list(sent_df.iterrows())):
        sent_text = row['text']
        sent_labels = eval(row['labels'])
        for name in names[:5]:
            if not name:
                continue
            t, l = update_name(sent_text, sent_labels, name)
            school_id.append(1)
            dt = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            created_at.append(dt)
            updated_at.append(dt)
            texts.append(t)
            labels.append(l)
    # Generate sample data
    sample_df = pd.DataFrame({
        "scool_id": school_id,
        "created_at": created_at,
        "updated_at": updated_at,
        "text": texts
    })
    truth_df = pd.DataFrame({
        "text": texts,
        "labels": labels
    })
    return {"sample": sample_df, "truth": truth_df}

In [12]:
sent_csv = "kwdlc_knp/sentences.csv"
name_csv = "names/full_names.csv"
for name_style in ("kanji", "katakana", "hiragana", "roma"):
    df = generate_data(sent_csv, name_csv, name_style)
    df["truth"].to_csv(f"name_{name_style}_truth_data.csv")
    df["sample"].to_csv(f"name_{name_style}_sample_data.csv")

100%|██████████| 983/983 [00:00<00:00, 7010.26it/s]


In [14]:
!head *.csv

==> name_hiragana_sample_data.csv <==
,scool_id,created_at,updated_at,text
0,1,2022-03-28 18:25:27,2022-03-28 18:25:27,布川事件を描いたドキュメンタリー映画「さとうはすとさとうはす」の試写会に行ってきました。
1,1,2022-03-28 18:25:27,2022-03-28 18:25:27,布川事件を描いたドキュメンタリー映画「さとうさくらとタさとうさくらの試写会に行ってきました。
2,1,2022-03-28 18:25:27,2022-03-28 18:25:27,布川事件を描いたドキュメンタリー映画「さとうみさきとタカオさとうみさき写会に行ってきました。
3,1,2022-03-28 18:25:27,2022-03-28 18:25:27,布川事件を描いたドキュメンタリー映画「さとうみさきとタカオ」のさとうみさきに行ってきました。
4,1,2022-03-28 18:25:27,2022-03-28 18:25:27,布川事件を描いたドキュメンタリー映画「さとうふうたとタカオ」の試写さとうふうたってきました。
5,1,2022-03-28 18:25:27,2022-03-28 18:25:27,東京・霞が関の国交省で、さとうはす政務官からさとうはす高梁市長に認定証が手渡された。
6,1,2022-03-28 18:25:27,2022-03-28 18:25:27,東京・霞が関の国交省で、さとうさくら政務官から近さとうさくら梁市長に認定証が手渡された。
7,1,2022-03-28 18:25:27,2022-03-28 18:25:27,東京・霞が関の国交省で、さとうみさき政務官から近藤隆さとうみさき長に認定証が手渡された。
8,1,2022-03-28 18:25:27,2022-03-28 18:25:27,東京・霞が関の国交省で、さとうみさき政務官から近藤隆則高さとうみさき認定証が手渡された。

==> name_hiragana_truth_data.csv <==
,text,labels
0,布川事件を描いたドキュメンタリー映画「さとうはすとさとうはす」の試写会に行ってきました。,"[(19, 24), (25, 30)